In [10]:
from awpy import DemoParser
import matplotlib.pyplot as plt
import os
import pandas as pd
pd.set_option('display.max_columns', None)



####### 
# Write the output name for the csv file containing the dataframe
#######

output_name = 'vita-vs-heroic-dataframe'

### Use this to create a dataframe with all the files in the `demos_rar` folder

In [ ]:
## Get all file names
demos = os.listdir('demos_rar')
demos = [d for d in demos if d.endswith('.dem')]


for file in demos:
    print(f"demos_rar/{file}")

### Manual selection of a single demo for predicting a match round by round

In [13]:
demos = ['vitality-vs-heroic-m1-mirage.dem']

In [14]:
full_dataframe = pd.DataFrame()

for file in demos:
    demo_parser = DemoParser(demofile=f"demos_rar/{file}", parse_rate=128)
    print(f'parsing thru {file}')
    df = demo_parser.parse(return_type='df')
    player_frames = df['playerFrames']
    rounds = df['rounds']
    
    score = rounds[['roundNum','tScore','ctScore', 'winningSide']]
    score_dict = score.set_index('roundNum')[['tScore', 'ctScore', 'winningSide']].to_dict()

    print(f'clearing unused columns')
    columns_drop = ['name', 'ping', 'teamName', 'team', 'cashSpendTotal',
    'x', 'y', 'z', 'eyeX', 'eyeY', 'eyeZ', 'velocityX', 'velocityY', 'velocityZ',
    'viewX', 'viewY', 'steamID', 'activeWeapon', 'isBot', 'lastPlaceName',
    'isBlinded', 'isAirborne', 'isDucking', 'isDuckingInProgress',
        'lastPlaceName', 'isUnDuckingInProgress', 'isReloading',
        'isInBombZone', 'isInBuyZone', 'isStanding', 'isScoped',
        'isWalking', 'isUnknown', 'spotters', 'zoomLevel', 'matchID', 'totalUtility']
    cleaned_frames = player_frames.drop(columns_drop, axis=1)
    # columns_drop
    columns_teams = ['side', 'hp', 'armor', 'flashGrenades', 'smokeGrenades',
    'heGrenades', 'fireGrenades', 'isAlive', 'isDefusing',
    'isPlanting', 'equipmentValue', 'equipmentValueFreezetimeEnd',
    'equipmentValueRoundStart', 'cash', 'cashSpendThisRound',
        'hasHelmet', 'hasDefuse', 'hasBomb']
    cleaned_common = cleaned_frames.drop(columns_teams, axis=1)
    common_columns = cleaned_common.columns.drop('tick')
    print(f'transforming multiple rows data into one')
    cleaned_ct = cleaned_frames[cleaned_frames['side'] == 'CT'].drop(common_columns, axis=1).add_prefix('ct_')
    cleaned_t = cleaned_frames[cleaned_frames['side'] == 'T'].drop(common_columns, axis=1).add_prefix('t_')
    grouped_common = cleaned_common.groupby('tick').first()
    grouped_ct  = cleaned_ct.groupby('ct_tick').sum(numeric_only=True)
    grouped_t  = cleaned_t.groupby('t_tick').sum(numeric_only=True)

    print(f'joining all data')
    tick_frames = pd.concat([grouped_common, grouped_ct, grouped_t], axis=1)

    tick_frames['t_Score'] = tick_frames['roundNum'].map(score_dict.get('tScore'))
    tick_frames['ct_Score'] = tick_frames['roundNum'].map(score_dict.get('ctScore'))
    tick_frames['winningSide'] = tick_frames['roundNum'].map(score_dict.get('winningSide'))
    full_dataframe = pd.concat([full_dataframe, tick_frames], axis=0)

    print(full_dataframe.shape)


### Change the name to the output
full_dataframe.to_csv(f'{output_name}.csv')

parsing thru vitality-vs-heroic-m1-mirage.dem
clearing unused columns
transforming multiple rows data into one
joining all data
(2320, 40)
